In [1]:
import os
import time
import math
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [14]:
#dir(Pair)

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
mint_filt = Filter.create_filter(address=None, event_types=[Pair.events.Mint])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 100

In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=mint_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['buyer'] = Conversion().convert_uint256_hex_string_to_address(topics[1])

    if(len(arguments) >= 1):
        event['details']['purchase_price'] = Conversion().convert_int256_bytes_to_int(arguments[0])
    else:
        event['details']['transfer_value'] = math.nan
     
    events[k] = event
    if key not in processed_events:
        print(f"Transfer at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Transfer at block:60,633,567 tx:0xb0cf7f5d664457b5a5e162c642d80954a121ac539081a9fffb2a2b1687ee8530
Transfer at block:60,633,567 tx:0xb0cf7f5d664457b5a5e162c642d80954a121ac539081a9fffb2a2b1687ee8530
Transfer at block:60,633,567 tx:0xb0cf7f5d664457b5a5e162c642d80954a121ac539081a9fffb2a2b1687ee8530
Transfer at block:60,633,569 tx:0xf8401a1d5b1e00faa6d9256a4f0991cebe54b0f2c51dfea3d2205a575065ec20
Transfer at block:60,633,569 tx:0xf8401a1d5b1e00faa6d9256a4f0991cebe54b0f2c51dfea3d2205a575065ec20
Transfer at block:60,633,569 tx:0xf8401a1d5b1e00faa6d9256a4f0991cebe54b0f2c51dfea3d2205a575065ec20
Transfer at block:60,633,573 tx:0x6088d7166c43d75349aebeaf3e3964d5d8e7de47eb82dab5e2e69ff9aaa8a3e2
Transfer at block:60,633,573 tx:0x6088d7166c43d75349aebeaf3e3964d5d8e7de47eb82dab5e2e69ff9aaa8a3e2
Transfer at block:60,633,573 tx:0x6088d7166c43d75349aebeaf3e3964d5d8e7de47eb82dab5e2e69ff9aaa8a3e2
Transfer at block:60,633,575 tx:0xd98216d696150f153abc582b70a5e257e1974732690aa41325e6ad9ca52596aa
Transfer a

In [8]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0x5c2ed810328349100a66b82b78a1791b101c9d61',
  'tx_hash': '0xb0cf7f5d664457b5a5e162c642d80954a121ac539081a9fffb2a2b1687ee8530',
  'blk_num': 60633567,
  'timestamp': 1723755565,
  'details': {'web3_type': web3._utils.datatypes.Mint,
   'buyer': '0xe8599F3cc5D38a9aD6F3684cd5CEa72f10Dbc383',
   'purchase_price': 50}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0x5c2ed810328349100a66b82b78a1791b101c9d61',
  'tx_hash': '0xb0cf7f5d664457b5a5e162c642d80954a121ac539081a9fffb2a2b1687ee8530',
  'blk_num': 60633567,
  'timestamp': 1723755565,
  'details': {'web3_type': web3._utils.datatypes.Mint,
   'buyer': '0x1d8b86e3D88cDb2d34688e87E72F388Cb541B7C8',
   'purchase_price': 3451992}},
 2: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0x1a13f4ca1d028320a707d995

In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,mint,sushi,0x5c2ed810328349100a66b82b78a1791b101c9d61,0xb0cf7f5d664457b5a5e162c642d80954a121ac539081...,60633567,1723755565,{'web3_type': <class 'web3._utils.datatypes.Mi...
1,polygon,uniswapv2pair,mint,sushi,0x5c2ed810328349100a66b82b78a1791b101c9d61,0xb0cf7f5d664457b5a5e162c642d80954a121ac539081...,60633567,1723755565,{'web3_type': <class 'web3._utils.datatypes.Mi...
2,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0xb0cf7f5d664457b5a5e162c642d80954a121ac539081...,60633567,1723755565,{'web3_type': <class 'web3._utils.datatypes.Mi...
3,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0xf8401a1d5b1e00faa6d9256a4f0991cebe54b0f2c51d...,60633569,1723755571,{'web3_type': <class 'web3._utils.datatypes.Mi...
4,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0xf8401a1d5b1e00faa6d9256a4f0991cebe54b0f2c51d...,60633569,1723755571,{'web3_type': <class 'web3._utils.datatypes.Mi...
5,polygon,uniswapv2pair,mint,sushi,0x28424507fefb6f7f8e9d3860f56504e4e5f5f390,0xf8401a1d5b1e00faa6d9256a4f0991cebe54b0f2c51d...,60633569,1723755571,{'web3_type': <class 'web3._utils.datatypes.Mi...
6,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0x6088d7166c43d75349aebeaf3e3964d5d8e7de47eb82...,60633573,1723755579,{'web3_type': <class 'web3._utils.datatypes.Mi...
7,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0x6088d7166c43d75349aebeaf3e3964d5d8e7de47eb82...,60633573,1723755579,{'web3_type': <class 'web3._utils.datatypes.Mi...
8,polygon,uniswapv2pair,mint,sushi,0x28424507fefb6f7f8e9d3860f56504e4e5f5f390,0x6088d7166c43d75349aebeaf3e3964d5d8e7de47eb82...,60633573,1723755579,{'web3_type': <class 'web3._utils.datatypes.Mi...
9,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0xd98216d696150f153abc582b70a5e257e1974732690a...,60633575,1723755583,{'web3_type': <class 'web3._utils.datatypes.Mi...
